<a href="https://colab.research.google.com/github/NehaKumari500092077/Machine-Learning-Lab/blob/main/Session3_Assignments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Assignment - 2**

####  **Objective** - Classify emails as spam or not spam using the [Spambase](https://archive.ics.uci.edu/static/public/94/data.csv) dataset, which contains the frequency of occurrence of 57 entities (words and characters) (in %) from 4601 spam and non-spam emails and corresponding class labels.  

Spam Classification with Logistic Regression







<font color='blue'>Task 4: Check Dataset Balance</font> [Marks - 1]

- Determine if the dataset is balanced or imbalanced by analyzing the distribution of the target variable.

<font color='blue'>Task 5: Split the Data</font> [Marks - 1]

- Split the data into training, validation, and test sets with the following proportions:
  - Training set: 70%
  - Validation set: 15%
  - Test set: 15%


<font color='blue'>Task 6: Implement Logistic Regression from Scratch</font> [Marks - 8]

- Implement a logistic regression model from scratch using:
  - Cross-entropy loss as the cost function.
  - Gradient descent as the learning algorithm.
  - L1 regularization in the cost function.
- Train the logistic regression model using gradient descent.
- Choose appropriate values for the learning rate and regularization parameter through cross-validation.

<font color='blue'>Task 7: Evaluate Model Performance</font> [Marks - 2]

- Evaluate the performance of your logistic regression model on the test set using:
  - Confusion matrix.
  - Precision.
  - Recall.
  - Area under the Precision-Recall Curve (AUC-PR).


<font color='blue'>Task 8: Compare with Scikit-Learn</font> [Marks - 5]

- Fit a logistic regression model using scikit-learn.
- Compare the performance of your implementation with the scikit-learn model by evaluating:
  - Confusion matrix.
  - Precision.
  - Recall.
  - Area under the Precision-Recall Curve (AUC-PR).
- Compare the AUC-PR values between your implementation and scikit-learn's implementation.

In [1]:
!pip install ucimlrepo

In [3]:
# from ucimlrepo import fetch_ucirepo

# # fetch dataset
# spambase = fetch_ucirepo(id=94)

# # data (as pandas dataframes)
# X = spambase.data.features
# y = spambase.data.targets

# # metadata
# print(spambase.metadata)

# # variable information
# print(spambase.variables)

<font color='blue'>Task 1: Load the Spambase Dataset</font> [Marks - 1]

- Load the Spambase dataset using Pandas.
- Display the first few rows to understand the structure of the data.

In [34]:
import pandas as pd

df = pd.read_csv("https://archive.ics.uci.edu/static/public/94/data.csv")
dataset = pd.DataFrame(df)
dataset.head(2)

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,Class
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.0,0.00,...,0.0,0.000,0.0,0.778,0.00,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.0,0.94,...,0.0,0.132,0.0,0.372,0.18,0.048,5.114,101,1028,1


<font color='blue'>Task 2: Check for Duplicate Entries and Missing Values</font> [Marks - 1]

- Check if there are duplicate entries in the dataset.
- Remove any duplicate entries.
- Check for missing values in the dataset.
- Remove entries with missing feature values.

In [21]:
#total 391 duplicate entries
duplicateEnteris = dataset.duplicated().sum()
print(duplicateEnteris)

391


In [27]:
#remove all the duplicate row except first
dataset.drop_duplicates(inplace=True)

In [28]:
dataset.duplicated().sum()

0

In [33]:
# Check for missing values in the DataFrame
#no missing values
pd.isnull(dataset).sum()




word_freq_make                0
word_freq_address             0
word_freq_all                 0
word_freq_3d                  0
word_freq_our                 0
word_freq_over                0
word_freq_remove              0
word_freq_internet            0
word_freq_order               0
word_freq_mail                0
word_freq_receive             0
word_freq_will                0
word_freq_people              0
word_freq_report              0
word_freq_addresses           0
word_freq_free                0
word_freq_business            0
word_freq_email               0
word_freq_you                 0
word_freq_credit              0
word_freq_your                0
word_freq_font                0
word_freq_000                 0
word_freq_money               0
word_freq_hp                  0
word_freq_hpl                 0
word_freq_george              0
word_freq_650                 0
word_freq_lab                 0
word_freq_labs                0
word_freq_telnet              0
word_fre

In [36]:
#Remove entries with missing feature values.
dataset.dropna(inplace=True)


<font color='blue'>Task 3: Preprocess the Dataset</font> [Marks - 1]

- Preprocess the dataset as required:
- Perform feature scaling or standardization on the features.